
## Content
*   GPT3 with zero-shot
    - observation
*   GPT3 with few shots
    - prepare data without diet type or allergens
    - prepare data with diet type and allergens
    - GPT3 Fewshot without diet types or allergens
    - GPT3 Fewshot with diet types and allergens
*   W&B Hyperparameter Tuning with Fewshot GPT3
*   Next Step: Prepare Dataset for Fine-Tuning









In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
df = pd.read_csv("drive/MyDrive/NLP244FinalProject/Data/RecipeNLG_dataset.csv")

In [ ]:
recipe_df = df[['title', 'ingredients', 'directions', 'NER']]

In [ ]:
recipe_df.head()

,title,ingredients,directions,NER
0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...","[""brown sugar"", ""milk"", ""vanilla"", ""nuts"", ""bu..."
1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....","[""beef"", ""chicken breasts"", ""cream of mushroom..."
2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...","[""frozen corn"", ""cream cheese"", ""butter"", ""gar..."
3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...","[""chicken"", ""chicken gravy"", ""cream of mushroo..."
4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...","[""peanut butter"", ""graham cracker crumbs"", ""bu..."


In [ ]:
# prepare data for cohere 
import json
path_data = "/content/drive/MyDrive/NLP244FinalProject/Data/"
jsonl_file = 'training_data_prepared.jsonl'
txt_file = 'training_data_prepared.txt'

with open(path_data+jsonl_file, 'r') as f_in, open(path_data+txt_file, 'w') as f_out:
    for line in f_in:
        json_obj = json.loads(line)
        prompt = json_obj['prompt']
        completion = json_obj['completion']
        f_out.write(f'{prompt}\n{completion}\n--SEPARATOR--\n')

In [ ]:
!pip install --upgrade pip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 65.6 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 22.0.4
    Uninstalling pip-22.0.4:
      Successfully uninstalled pip-22.0.4


In [ ]:
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached openai-0.27.1.tar.gz (57 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 35.1 MB/s eta 0:00:00
  Created wheel for openai: filename=openai-0.27.1-py3-none-any.whl size=70091 sha256=85c970991732787dc2990f58ddd7221f42aedc3b755fc4965459d39f51566cf9
  Stored in directory: /root/.cache/pip/wheels/1f/d1/75/8015df8f7ec8ba5422d8a45786cbb64d421872f488c09303fe
Successfully built

In [ ]:
OPENAI_API_KEY = 'your_open_ai_api_key'

In [ ]:
import openai
openai.api_key = OPENAI_API_KEY

#### GPT3 with zero-shot
- before adjusting hyperparameters

In [ ]:
# define some example ingredients to generate recipe
example_ingredients1 = ["butter", "salt", "bacon", "milk", "flour"]
example_ingredients2 = ["peanutbutter", 'soysauce', 'bread', 'ramen', 'tofu', 'mushroom', 'milk']
example_ingredients3 = ['eggplant', 'tofu','mayonnaise', 'ketchup', 'noodles', 'green onions']

In [ ]:
# this is a zero shot example 
class GPT3RecipeGenerator:
    def __init__(self, api_key, model_engine="text-davinci-003"):
        self.model_engine = model_engine
        openai.api_key = api_key
        
    def generate_recipe(self, ingredients):
        prompt = f"Generate a recipe that contains {', '.join(ingredients)}:"
        response = openai.Completion.create(
            engine=self.model_engine,
            prompt=prompt,
            max_tokens=100,
            n=1, # "n" refers to the number of transformer layers used in the model.
            stop=None,
            temperature=0.5,
        )
        recipe = response["choices"][0]["text"].strip()
        return recipe

In [ ]:
recipe_generator = GPT3RecipeGenerator(OPENAI_API_KEY)
example_recipe1 = recipe_generator.generate_recipe(example_ingredients1)
example_recipe2 = recipe_generator.generate_recipe(example_ingredients2)
example_recipe3 = recipe_generator.generate_recipe(example_ingredients3)

In [ ]:
print("Zero-shot with default hypereparemters:", example_recipe1)

Zero-shot with default hypereparemters: Bacon and Cheese Biscuits

Ingredients:

-2 cups all-purpose flour
-1 teaspoon baking powder
-1 teaspoon baking soda
-1 teaspoon salt
-1/2 cup cold butter, cut into small cubes
-1/2 cup cooked bacon, chopped
-1/2 cup milk
-1 cup shredded cheddar cheese

Instructions:

1. Preheat oven to 400°F.

2. In a


In [ ]:
print("Zero-shot with default hypereparemters:", example_recipe2)

Zero-shot with default hypereparemters: Peanut Butter Ramen Bowl

Ingredients:

- 2 cups cooked ramen noodles
- 2 tablespoons peanut butter
- 1 tablespoon soysauce
- 2 slices of bread, toasted
- 1/2 block of tofu, cubed
- 1/2 cup sliced mushrooms
- 1/2 cup of milk

Instructions:

1. In a large pot, bring 4 cups of water to a boil.

2. Add the


In [ ]:
print("Zero-shot with default hypereparemters:", example_recipe3)

Zero-shot with default hypereparemters: Eggplant and Tofu Noodle Bowl

Ingredients: 

- 1 large eggplant, cut into cubes
- 1 block of firm tofu, cubed
- 2 tablespoons of mayonnaise
- 2 tablespoons of ketchup
- 4 cups of cooked noodles
- 2 green onions, sliced

Instructions:

1. Preheat oven to 375°F.

2. Arrange the eggplant cubes on a baking sheet. Bake


- after adjusting the hyperparameters
  - max tokens: 
    - 2048
    - generate a response that is at most 2048 tokens long
  - temperatue: 
    - 0.8
    - a higher temperature setting will result in more unpredictable and diverse responses, with a higher likelihood of generating unexpected or unusual language.

In [ ]:
# this is a zero shot example with modified hyperparameters
class GPT3RecipeGenerator2:
    def __init__(self, api_key, model_engine="text-davinci-003"):
        self.model_engine = model_engine
        openai.api_key = api_key
        
    def generate_recipe(self, ingredients):
        prompt = f"Generate a recipe that contains {', '.join(ingredients)}:"
        response = openai.Completion.create(
            engine=self.model_engine,
            prompt=prompt,
            max_tokens=2048,
            n=1,
            stop=None,
            temperature=0.8,
        )
        recipe = response["choices"][0]["text"].strip()

        return recipe

In [ ]:
recipe_generator2 = GPT3RecipeGenerator2(OPENAI_API_KEY)
example_recipe1_new = recipe_generator2.generate_recipe(example_ingredients1)
example_recipe2_new = recipe_generator2.generate_recipe(example_ingredients2)
example_recipe3_new = recipe_generator2.generate_recipe(example_ingredients3)

In [ ]:
print("Zero-shot after modifying hypereparemters:", example_recipe1_new)

Zero-shot after modifying hypereparemters: Bacon Cheddar Drop Biscuits

Ingredients:

- 2 cups all-purpose flour
- 1 tablespoon baking powder
- 1 teaspoon baking soda
- 1 teaspoon salt
- 6 tablespoons cold butter, cubed
- 4 slices cooked bacon, crumbled
- 1 cup sharp cheddar cheese, shredded
- 1 cup buttermilk

Instructions:

1. Preheat oven to 425°F. Line a baking sheet with parchment paper.

2. In a large bowl, whisk together the flour, baking powder, baking soda, and salt.

3. Add the butter cubes and use a pastry blender to cut in the butter until the mixture resembles coarse crumbs.

4. Stir in the crumbled bacon and shredded cheese until well combined.

5. Make a well in the center of the dry ingredients, pour the buttermilk into the well, and stir until just combined (mixture will be wet and sticky).

6. Drop dough by heaping tablespoons onto the prepared baking sheet, about 2 inches apart.

7. Bake for 12-15 minutes, or until golden brown. Serve warm. Enjoy!


In [ ]:
print("Zero-shot after modifying hypereparemters:", example_recipe2_new)

Zero-shot after modifying hypereparemters: Veggie Ramen Bowl

Ingredients:

- 2 packages of ramen noodles
- 2 tablespoons peanut butter
- 2 tablespoons soy sauce
- 2 slices of bread, toasted and cut into cubes
- 4 ounces firm tofu, cubed
- 4 ounces mushrooms, sliced
- 2 cups milk

Instructions:

1. Cook ramen noodles according to package instructions and set aside.

2. Heat a large skillet over medium-high heat and add the peanut butter, soy sauce, and bread cubes. Cook until the bread cubes are lightly browned.

3. Add the tofu and mushrooms and cook for 5 minutes.

4. Pour in the milk and bring to a boil.

5. Reduce heat and simmer for 10 minutes.

6. Add the cooked ramen noodles and stir until heated through.

7. Serve warm. Enjoy!


In [ ]:
print("Zero-shot after modifying hypereparemters:", example_recipe3_new)

Zero-shot after modifying hypereparemters: Eggplant Tofu Noodle Bowl

Ingredients:

-1 large eggplant, diced
-1 package of extra firm tofu, drained and cut into cubes
-3 cloves of garlic, minced
-1/4 cup mayonnaise
-2 tablespoons ketchup
-2 tablespoons sesame oil
-2 tablespoons soy sauce
-8 ounces of rice noodles
-2 green onions, sliced

Instructions:

1. Heat a large skillet over medium-high heat. Add the eggplant and tofu and cook until golden brown, about 5 minutes.

2. Add the garlic and cook for an additional minute.

3. In a small bowl, mix together the mayonnaise, ketchup, sesame oil, and soy sauce.

4. Add the noodles to the skillet and pour the sauce over top. Toss everything together and cook until the noodles are tender, about 5 minutes.

5. Garnish with the green onions and serve warm. Enjoy!


##### Observation
In the generated recipe after changing hyperparameters, the generated recipe contains ingredients not inside the input ingredients. So we may need to change the following for few shot learning to prevent that happen:
- change the prompt to: "Generate a recipe that only contains:..."
- change hyperparameters 'top_p', 'presence_penalty', 'frequency_penalty'
  - top_p: 
    - defualt: 1.0
    - The model will only generate tokens that have a probability higher than the threshold specified by this parameter. Increasing this value can result in more diverse and interesting text, but may also increase the likelihood of the model generating nonsensical text.
  - frequency_penalty: 
    - default: 0.0
    - which penalizes the model for generating tokens with low frequency in the training data. By setting a high frequency_penalty, the model is less likely to generate rare ingredients that were not mentioned in the input ingredients.
  - temperature
    - default:1.0
    - controls the level of randomness in the model's output. By setting a low temperature, the model will generate more conservative and predictable output. By setting a high temperature, the model will generate more creative and unpredictable output.

#### GPT3 with Few shots

##### Prepare data without diet type or allergens

In [ ]:
import json

In [ ]:
df_example = recipe_df[:10]

In [ ]:
import pandas as pd
fewshot_examples = []
for i, row in df_example.iterrows():
    fewshot_examples.append({
        "title": row["title"],
        "ingredients": row["NER"],
        "directions": row["directions"]
    })

##### Prepare data with diet type and allergens

In [ ]:
# prepare examples with diet type and allergy type with 5 examples
df_example_restrictions = recipe_df[:5]

In [ ]:
fewshot_examples_restrictions = []
for i, row in df_example_restrictions.iterrows():
    fewshot_examples_restrictions.append({
        "title": row["title"],
        "ingredients": row["NER"],
        "directions": row["directions"]
    })

In [ ]:
# label allergy and diet type specified fewshot examples 
def label_diet_types_allergy(recipes):
    diet_type = ['vegeterian', 'none', 'vegetarian', 'none', 'vegetarian']
    allergy_type = ['tree nuts, dairy', 'none', 'none', 'none', 'peanuts, dairy']
    new_recipes = []
    for i in range(len(recipes)):
      recipe = recipes[i]
      recipe['diet_type'] = diet_type[i]
      recipe['allergy_type'] = allergy_type[i]
      new_recipes.append(recipe)
    return new_recipes

In [ ]:
new_fewshot_examples_restrictions = label_diet_types_allergy(fewshot_examples_restrictions)

In [ ]:
new_fewshot_examples_restrictions

[{'title': 'No-Bake Nut Cookies',
  'ingredients': '["brown sugar", "milk", "vanilla", "nuts", "butter", "bite size shredded rice biscuits"]',
  'directions': '["In a heavy 2-quart saucepan, mix brown sugar, nuts, evaporated milk and butter or margarine.", "Stir over medium heat until mixture bubbles all over top.", "Boil and stir 5 minutes more. Take off heat.", "Stir in vanilla and cereal; mix well.", "Using 2 teaspoons, drop and shape into 30 clusters on wax paper.", "Let stand until firm, about 30 minutes."]',
  'diet_type': 'vegeterian',
  'allergy_type': 'tree nuts, dairy'},
 {'title': "Jewell Ball'S Chicken",
  'ingredients': '["beef", "chicken breasts", "cream of mushroom soup", "sour cream"]',
  'directions': '["Place chipped beef on bottom of baking dish.", "Place chicken on top of beef.", "Mix soup and cream together; pour over chicken. Bake, uncovered, at 275\\u00b0 for 3 hours."]',
  'diet_type': 'none',
  'allergy_type': 'none'},
 {'title': 'Creamy Corn',
  'ingredients':

In [ ]:
new_fewshot_examples_restrictions[0]

{'title': 'No-Bake Nut Cookies',
 'ingredients': '["brown sugar", "milk", "vanilla", "nuts", "butter", "bite size shredded rice biscuits"]',
 'directions': '["In a heavy 2-quart saucepan, mix brown sugar, nuts, evaporated milk and butter or margarine.", "Stir over medium heat until mixture bubbles all over top.", "Boil and stir 5 minutes more. Take off heat.", "Stir in vanilla and cereal; mix well.", "Using 2 teaspoons, drop and shape into 30 clusters on wax paper.", "Let stand until firm, about 30 minutes."]',
 'diet_type': 'vegeterian',
 'allergy_type': 'tree nuts, dairy'}

Hyperparameters for GPT3Fewshot
- To generate recipe: 
  - max_tokens: 2048
  - n:1
  - temperature=0.8
  - presence_penalty=0.8
- To tune with fewshot learning:
  - max_tokens:1024
  - n:1
  - temperature=0.5
  - presence_penalty=0.8

##### GPT3 Fewshot without diet type or allergens

In [ ]:
class GPT3RecipeGeneratorFewShot:
    def __init__(self, api_key, model_engine="text-davinci-003"):
        self.model_engine = model_engine
        openai.api_key = api_key
        
    def generate_recipe(self, ingredients):
        prompt = f"Generate a recipe that only contains {', '.join(ingredients)}:"
        response = openai.Completion.create(
            engine=self.model_engine,
            prompt=prompt,
            max_tokens=2048,
            n=1,
            stop=None,
            temperature=0.8,
            presence_penalty=0.8 # penalize recipe with ingredients not exist in the prompt
        )
        recipe = response["choices"][0]["text"].strip()

        return recipe
        
    def fine_tune(self, fewshot_examples):
        examples = []
        for example in fewshot_examples:
            prompt = f"Recipe: {example['title']}\nIngredients: {', '.join(example['ingredients'])}\n"
            prompt += f"Directions: {', '.join(example['directions'])}\n\n"
            examples.append(prompt)
        openai.Completion.create(
            model=self.model_engine,
            prompt=examples,
            temperature=0.5,
            max_tokens=1024,
            n=1,
            stop=None,
            presence_penalty=0.8 
        )


In [ ]:
# Create an instance of the GPT3RecipeGenerator class
fewshot_generator = GPT3RecipeGeneratorFewShot(OPENAI_API_KEY)
# Fine-tune the model with the few-shot examples
fewshot_generator.fine_tune(fewshot_examples)

In [ ]:
fewshot_recipe1 = fewshot_generator.generate_recipe(example_ingredients1)
fewshot_recipe2 = fewshot_generator.generate_recipe(example_ingredients2)
fewshot_recipe3 = fewshot_generator.generate_recipe(example_ingredients3)

In [ ]:
print(fewshot_recipe1)

Bacon and Cheese Waffles

Ingredients:

- 6 slices of bacon, cooked and crumbled
- 1/2 cup melted butter
- 3/4 cup all-purpose flour
- 1 teaspoon salt
- 1 1/2 cups milk
- 2 tablespoons butter, melted
- 1/2 cup finely grated cheddar cheese

Instructions:

1. Preheat a waffle iron to medium-high heat.

2. In a large bowl, whisk together the flour, salt, and melted butter until blended.

3. Slowly add in the milk, stirring until combined.

4. Gently fold in the crumbled bacon and grated cheese.

5. Grease the waffle iron with the remaining butter.

6. Spoon in the batter, spreading evenly to fill the waffle iron.

7. Cook for about 8 minutes, or until golden brown.

8. Serve with butter and syrup, if desired. Enjoy!


In [ ]:
print(fewshot_recipe2)

Peanut Butter Ramen Bowl

Ingredients: 
- 2 tablespoons of peanut butter
- 1 tablespoon of soysauce
- 2 slices of bread
- 2 packages of ramen noodles
- 1/2 block of firm tofu, cubed
- 8 ounces of sliced mushrooms
- 1 cup of milk

Instructions:

1. Bring a large pot of water to a boil and add the ramen noodles. Boil for 3 minutes, then drain the noodles and set aside.

2. In a large skillet over medium heat, heat some oil and add the cubed tofu. Cook until lightly browned, about 7-8 minutes.

3. Add the mushrooms and cook for an additional 5 minutes.

4. Stir in the peanut butter, soysauce, and milk. Simmer for a few minutes, stirring occasionally.

5. To assemble the bowl, place the noodles in a bowl and top with the tofu and mushroom mixture.

6. Garnish with a slice of bread and enjoy!


In [ ]:
print(fewshot_recipe3)

Eggplant Tofu Noodle Bowl
Ingredients:

- 1 large eggplant, diced
- 1 block extra-firm tofu, drained and cut into cubes
- 2 tablespoons mayonnaise
- 2 tablespoons ketchup
- 4 ounces noodles of your choice (rice, soba, etc.)
- 2 green onions, thinly sliced
- Salt and pepper to taste

Instructions:

1. Preheat oven to 375°F.

2. In a medium bowl, combine the diced eggplant, cubed tofu, and mayonnaise. Mix until all of the eggplant and tofu are evenly coated in the mayonnaise.

3. Spread the eggplant and tofu mixture onto a baking sheet and bake for 20 minutes or until golden brown.

4. Meanwhile, bring a large pot of water to a boil and cook noodles according to package instructions.

5. Once the eggplant and tofu are finished baking, remove from the oven and set aside.

6. When the noodles are done cooking, drain and rinse with cold water.

7. Place the noodles in a large bowl and add the eggplant and tofu.

8. Add the ketchup, green onions, and season with salt and pepper to taste.

9.

##### GPT3 Fewshot with Diet Types and Allergy

In [ ]:
class GPT3RecipeGeneratorDiet:
    def __init__(self, api_key, model_engine="text-davinci-003"):
        self.model_engine = model_engine
        openai.api_key = api_key
        
    def generate_recipe(self, ingredients, diet_type=None, allergy_type=None):
        prompt = f"Generate a recipe that only contains {', '.join(ingredients)}"
        if diet_type:
            prompt += f" and is suitable for {', '.join(diet_type)} diet"
        if allergy_type:
            prompt += f" and is free of {', '.join(allergy_type)} allergens"
        prompt += ":"
        response = openai.Completion.create(
            engine=self.model_engine,
            prompt=prompt,
            max_tokens=2048,
            n=1,
            stop=None,
            temperature=0.8,
            presence_penalty=0.8 # penalize recipe with ingredients not exist in the prompt
        )
        recipe = response["choices"][0]["text"].strip()

        return recipe
        
    def fine_tune(self, fewshot_examples):
        examples = []
        for example in fewshot_examples:
            prompt = f"Recipe: {example['title']}\nIngredients: {', '.join(example['ingredients'])}\n"
            if 'diet_type' in example:
                prompt += f"Diet Type: {', '.join(example['diet_type'])}\n" 
            if 'allergy_type' in example:
                prompt += f"Allergy Type: {', '.join(example['allergy_type'])}\n"
            prompt += f"Directions: {', '.join(example['directions'])}\n\n"
            examples.append(prompt)
        openai.Completion.create(
            model=self.model_engine,
            prompt=examples,
            temperature=0.5,
            max_tokens=1024,
            n=1,
            stop=None,
            presence_penalty=0.8 
        )

# Create an instance of the GPT3RecipeGenerator class
fewshot_generator_diet = GPT3RecipeGeneratorDiet(OPENAI_API_KEY)
# Fine-tune the model with the few-shot examples
fewshot_generator_diet.fine_tune(new_fewshot_examples_restrictions)

In [ ]:
print(example_ingredients1)
print(example_ingredients2)
print(example_ingredients3)


['butter', 'salt', 'bacon', 'milk', 'flour']
['peanutbutter', 'soysauce', 'bread', 'ramen', 'tofu', 'mushroom', 'milk']
['eggplant', 'tofu', 'mayonnaise', 'ketchup', 'noodles', 'green onions']


In [ ]:
# add diet type to the original ingredients to experiment changes in recipe
fewshot_recipe1_diet = fewshot_generator_diet.generate_recipe(example_ingredients1, diet_type='keto')
fewshot_recipe2_diet = fewshot_generator_diet.generate_recipe(example_ingredients2, diet_type='keto', allergy_type='peanuts')
fewshot_recipe3_diet = fewshot_generator_diet.generate_recipe(example_ingredients3, diet_type='keto', allergy_type='gluten')

In [ ]:
print(fewshot_recipe1_diet)

Bacon and Chive Keto Pancakes

Ingredients:

- 4 slices bacon, diced
- 2 tablespoons butter, melted
- 1/2 cup all-purpose flour
- 1/2 teaspoon salt
- 1/2 cup milk
- 2 tablespoons fresh chives, chopped

Instructions:

1. Heat a large skillet over medium-high heat. Add the bacon and cook until crispy, about 5 minutes. Remove from the pan and set aside.

2. In a medium bowl, stir together the melted butter, flour, and salt. Whisk in the milk until the mixture is smooth. Stir in the bacon and chives.

3. Grease the skillet with a little butter. Drop heaping 1/4 cup portions of the batter into the skillet, spreading out slightly. Cook for 1-2 minutes on each side, or until golden brown.

4. Serve warm with your favorite toppings. Enjoy!


In [ ]:
print(fewshot_recipe2_diet)
# but we do not expect to see peanut in this bowl eventho the input ingredients with peanut

Vegan Peanut Butter Ramen Bowl

Ingredients:
- 2 cups low sodium vegetable broth 
- 2 tablespoons low sodium soy sauce
- 2 tablespoons peanut butter 
- 2 packets instant ramen noodles 
- 8 ounces extra firm tofu, cut into cubes 
- 1 teaspoon vegetable oil 
- 2 cups sliced mushrooms 
- ¼ cup non-dairy milk 
- 2 slices gluten free bread

Instructions:
1. In a large pot, bring the vegetable broth and soy sauce to a boil.
2. Add the instant ramen noodles, stir to combine, and cook according to package directions.
3. Heat the vegetable oil in a large skillet over medium heat. Add the cubed tofu and stir to coat. Cook for 5 minutes, stirring occasionally.
4. Add the mushrooms and cook for an additional 3 minutes.
5. Reduce the heat to low and add the peanut butter and non-dairy milk. Stir until the peanut butter is melted and evenly distributed.
6. Serve the ramen noodles in individual bowls and top with the peanut butter and mushroom mixture.
7. Serve with a side of gluten free bread. Enjoy

In [ ]:
print(fewshot_recipe3_diet)
# it specify "gluten free" in the noodles which is good!

Sesame Eggplant and Tofu Noodle Bowl

Ingredients:

-1 large eggplant, cut into cubes

-1 block of firm tofu, cut into cubes

-1 tablespoon vegetable oil

-3 cloves garlic, minced

-2 tablespoons sesame oil

-1/4 cup mayonnaise

-2 tablespoons ketchup

-2 bunches green onions, chopped

-8 ounces noodles of choice (Gluten Free)

Instructions:

1. Preheat oven to 400 degrees F.

2. Line a baking sheet with parchment paper.

3. Place the cubed eggplant and tofu on the prepared baking sheet. Drizzle with the oil and sprinkle with half of the minced garlic.

4. Roast in the preheated oven for 20 minutes, stirring halfway through.

5. Meanwhile, cook the noodles according to package instructions.

6. In a small bowl, combine the mayonnaise and ketchup.

7. Heat the remaining oil and garlic in a skillet over medium heat. When the garlic begins to sizzle, add the roasted eggplant and tofu. Cook, stirring frequently, for 5 minutes.

8. Add the cooked noodles, green onions and sesame oil to the 

#### W&B Hyperparameter Tuning with Fewshot GPT3

In [ ]:
!pip install wandb  # install W&B library

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 13.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 5.7 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=a678b876a05ee3858f897d7dbb9997f888c9e273e9c6b03805e7d3008e035b65
  Stored in directory: /root/.cache/pip/wheels/4c/8e/7e/72fbc243e1aeecae64a96875432e70d4e92f3d2d18123be004
Successfully built pathtools


In [ ]:
import wandb
import random

In [ ]:
wandb.login()  # key: 51cb3f2f19c8ab16c6a78b55bcc1237f8ee512b4

wandb: Currently logged in as: zpan34 (recipe_generation). Use `wandb login --relogin` to force relogin


True

In [ ]:
wandb.init(project='fewshot-gpt3')

In [ ]:
# prepare new few shot examples
def prepare_few_shot_examples(df_example):
    fewshot_examples = []
    for i, row in df_example.iterrows():
        fewshot_examples.append({
            "title": row["title"],
            "ingredients": row["NER"],
            "directions": row["directions"]
        })
    return fewshot_examples

In [ ]:
# Split recipe_df into training and validation sets
recipe_df_sm = recipe_df.sample(frac=0.0025, random_state=42)
print(recipe_df_sm.shape)
train_df = recipe_df_sm.sample(frac=0.8, random_state=42)
valid_df = recipe_df_sm.drop(train_df.index)
print(train_df.shape)
print(valid_df.shape)

(5578, 4)
(4462, 4)
(1116, 4)


In [ ]:
# due to per token rate limit, only take a few sample from train for fewshot...
fewshot_train_data = prepare_few_shot_examples(train_df[:5])

In [ ]:
fewshot_train_data[0]

{'title': "Italian Cream Cake(Mr. Hendrickson'S Favorite)  ",
 'ingredients': '["margarine", "Crisco", "sugar", "egg yolks", "all-purpose", "soda", "buttermilk", "coconut", "pecans", "egg whites"]',
 'directions': '["Using mixer, beat margarine and Crisco. Add sugar and beat well. Add egg yolks, one at a time. Sift flour and soda together and add alternately with buttermilk, still using mixer.", "Add vanilla, coconut and pecans. Fold in beaten egg whites.", "Pour into 3 greased and lined 8 or 9-inch cake pans.", "Bake at 350\\u00b0 for 45 minutes or until sides break loose from pan."]'}

In [ ]:
class GPT3RecipeGeneratorFewShot:
    def __init__(self, api_key, model_engine="text-davinci-003"):
        self.model_engine = model_engine
        openai.api_key = api_key
        
    def generate_recipe(self, ingredients, temperature=0.8, max_tokens=2048, presence_penalty=0.8):
        prompt = f"Generate a recipe that only contains {', '.join(ingredients)}:"
        response = openai.Completion.create(
            engine=self.model_engine,
            prompt=prompt,
            max_tokens=max_tokens,
            n=1,
            stop=None,
            temperature=temperature,
            presence_penalty=presence_penalty # penalize recipe with ingredients not exist in the prompt
        )
        recipe = response["choices"][0]["text"].strip()

        return recipe
        
    def fine_tune(self, fewshot_examples, temperature=0.5, max_tokens=2048, presence_penalty=0.8):
        examples = []
        for example in fewshot_examples:
            prompt = f"Recipe: {example['title']}\nIngredients: {', '.join(example['ingredients'])}\n"
            prompt += f"Directions: {', '.join(example['directions'])}\n\n"
            examples.append(prompt)
        openai.Completion.create(
            model=self.model_engine,
            prompt=examples,
            temperature=temperature,
            max_tokens=max_tokens,
            n=1,
            stop=None,
            presence_penalty=presence_penalty 
        )


In [ ]:
# initialize the generator 
fewshot_generator = GPT3RecipeGeneratorFewShot(OPENAI_API_KEY)

In [ ]:
hyperparameters = {
    "temperature": [0.5, 0.8, 0.9],
    "max_tokens": [1024, 1800, 2048],
    "presence_penalty": [0.2, 0.5, 0.8],
}

In [ ]:
print(valid_df.shape)

(1116, 4)


In [ ]:
valid_df_limit = valid_df[:100]

In [ ]:
valid_df.head()

,title,ingredients,directions,NER
1334975,Moms Pie Dough,"[""4.5 Cups Flour"", ""1.5 Tsp Salt"", ""Pinch Baki...","[""Mix all dry ingredients in a bowl."", """", ""Ad...","[""Flour"", ""Salt"", ""Baking Powder"", ""Sugar"", ""C..."
116562,Pretzel Salad Or Dessert,"[""2 c. crushed small thin pretzels (sticks)"", ...","[""Mix and press in baking pan, approximately 1...","[""thin pretzels"", ""margarine""]"
1712896,Citrus Syrup,"[""3/4 cup sugar"", ""1/2 cup fresh orange juice""...","[""In a 1 1/2-quart saucepan stir together suga...","[""sugar"", ""orange juice"", ""lemon juice""]"
633422,Dill Dip,"[""2/3 c. sour cream"", ""2/3 c. Hellmann's mayo""...","[""Mix all ingredients together and chill overn...","[""sour cream"", ""mayo"", ""parsley flakes"", ""onio..."
1073484,Portzelky (New Year'S Cookies),"[""1 teaspoon sugar"", ""1/2 cup lukewarm water"",...","[""Dissolve sugar in lukewarm water in a small ...","[""sugar"", ""water"", ""active dry yeast"", ""milk"",..."


In [ ]:
valid_df.shape

(1116, 4)

In [ ]:
valid_df_limit.shape

(100, 4)

In [ ]:
path_res = 'drive/MyDrive/NLP244FinalProject/GeneratedRecipes(ValidationData)/'

In [ ]:
valid_df_limit.to_csv(path_res+"reference_validation_recipes.csv", index=False)

In [ ]:
# Train the model using the training set and test the hyperparameters on the validation set
for i in range(1): # Number of runs
    # Select random hyperparameters
    hyperparams = {
        "temperature": random.choice(hyperparameters["temperature"]),
        "max_tokens": random.choice(hyperparameters["max_tokens"]),
        "presence_penalty": random.choice(hyperparameters["presence_penalty"]),
    }
    
    # Fine-tune the model using the training set and the selected hyperparameters
    fewshot_generator.fine_tune(fewshot_train_data, **hyperparams)
    
    # Generate a recipe with the selected hyperparameters on the validation set
    valid_recipes = []
    for i in range(len(valid_df_limit)):
        ingredients = valid_df_limit.iloc[i]['NER']
        recipe = fewshot_generator.generate_recipe(ingredients)
        valid_recipes.append(recipe)

    # Log the hyperparameters and generated recipes to W&B as a dataframe
    recipe_df = pd.DataFrame({'Ingredients': valid_df_limit['NER'], 'Generated Recipe': valid_recipes})
    recipe_df.to_csv(path_res+"recipes_hyp5.csv", index=False)
    wandb.log({'Hyperparameters': hyperparams, 'Generated Recipes': wandb.Table(dataframe=recipe_df)})

In [ ]:
# The logged dataformat in W&B will be like this:

# wandb.log({
#     "temperature": 0.8,
#     "max_tokens": 1024,
#     "presence_penalty": 0.8,
#     "recipe": "This is the generated recipe."
# })

In [ ]:
print(example_ingredients3)

['eggplant', 'tofu', 'mayonnaise', 'ketchup', 'noodles', 'green onions']


In [ ]:
# try an old example to test the difference 
print(example_ingredients3)
new_fewshot_recipe3 = fewshot_generator.generate_recipe(example_ingredients3)

['eggplant', 'tofu', 'mayonnaise', 'ketchup', 'noodles', 'green onions']


In [ ]:
print(new_fewshot_recipe3)

Eggplant Tofu Noodle Stir-Fry

Ingredients:

-1 large eggplant, cut into cubes
-1 (14 ounce) package extra-firm tofu, cubed
-2 tablespoons mayonnaise
-2 tablespoons ketchup
-4 ounces noodles (such as spaghetti or linguine)
-2 green onions, sliced
-Salt and pepper to taste

Instructions:

1. Heat a large skillet over medium-high heat and add the eggplant. Cook for about 5 minutes, stirring often, until the eggplant is lightly browned and beginning to soften.

2. Add the tofu and cook for another 5 minutes, stirring occasionally.

3. Add the mayonnaise and ketchup and stir until everything is evenly coated.

4. Add the noodles and green onions and cook until the noodles are al dente, about 5 minutes.

5. Season with salt and pepper to taste and serve.


### Next Step: Prepare Dataset for Fine-Tuning

In [ ]:
recipe_df.head()

,title,ingredients,directions,NER
0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...","[""brown sugar"", ""milk"", ""vanilla"", ""nuts"", ""bu..."
1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....","[""beef"", ""chicken breasts"", ""cream of mushroom..."
2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...","[""frozen corn"", ""cream cheese"", ""butter"", ""gar..."
3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...","[""chicken"", ""chicken gravy"", ""cream of mushroo..."
4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...","[""peanut butter"", ""graham cracker crumbs"", ""bu..."


In [ ]:
recipe_df.shape

(2231142, 4)

In [ ]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(recipe_df, test_size=0.5, random_state=42)

In [ ]:
train_df.shape

(1115571, 4)

In [ ]:
# prepare jsonl format training data 
import json 

def prepare_data(df):
    data = []
    for index, row in df.iterrows():
      title = row['title']
      ingredients = eval(row['NER'])
      directions = eval(row['directions'])
      prompt = f"Generate a recipe that only contains ingredients: {', '.join(ingredients)}"
      directions_string = ",".join(directions)
      completion = f"Title: {title}\n\n Directions: {directions_string}"
      data.append({"prompt": prompt, "completion": completion})
    return data


In [ ]:
training_data_example = prepare_data(df_example)

In [ ]:
training_data_example[:2]

[{'prompt': 'Generate a recipe that only contains ingredients: brown sugar, milk, vanilla, nuts, butter, bite size shredded rice biscuits',
  'completion': 'Title: No-Bake Nut Cookies\n\n Directions: In a heavy 2-quart saucepan, mix brown sugar, nuts, evaporated milk and butter or margarine.,Stir over medium heat until mixture bubbles all over top.,Boil and stir 5 minutes more. Take off heat.,Stir in vanilla and cereal; mix well.,Using 2 teaspoons, drop and shape into 30 clusters on wax paper.,Let stand until firm, about 30 minutes.'},
 {'prompt': 'Generate a recipe that only contains ingredients: beef, chicken breasts, cream of mushroom soup, sour cream',
  'completion': "Title: Jewell Ball'S Chicken\n\n Directions: Place chipped beef on bottom of baking dish.,Place chicken on top of beef.,Mix soup and cream together; pour over chicken. Bake, uncovered, at 275° for 3 hours."}]

In [ ]:
path_data = '/content/drive/My Drive/NLP244FinalProject/Data/'

In [ ]:
training_data = prepare_data(train_df)

# Write data to file
with open(path_data+'training_data.jsonl', 'w') as f:
    for data_point in training_data:
        f.write(json.dumps(data_point) + '\n')